In [3]:
print("yugor")

yugor


In [23]:
import os
os.chdir("../")

In [24]:
%pwd

'c:\\Users\\nhant\\Projects\\Medical-chatbot'

In [25]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [26]:
def load_pdf_files(data):
    loader = DirectoryLoader(data, glob="*.pdf",
                             loader_cls=PyPDFLoader)
  
    documents = loader.load()
    return documents

In [27]:
extracted_data = load_pdf_files("data")

In [28]:
len(extracted_data)

428

In [29]:
extracted_data

[Document(metadata={'producer': 'Pdftools SDK', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2024-11-18T23:22:09+00:00', 'source': 'data\\Human-Physiology.pdf', 'total_pages': 428, 'page': 0, 'page_label': '1'}, page_content='LECTURE NOTES \n \nFor Nursing Students \n \n \n \n \n \nHuman Anatomy \nand Physiology \n \n \n \n \n \n \n \n \nNega Assefa \nAlemaya University \nYosief Tsige \nJimma University \n \n \nIn collaboration with the Ethiopia Public Health Training Initiative, The Carter Center, \nthe Ethiopia Ministry of Health, and the Ethiopia Ministry of Education \n \n2003'),
 Document(metadata={'producer': 'Pdftools SDK', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2024-11-18T23:22:09+00:00', 'source': 'data\\Human-Physiology.pdf', 'total_pages': 428, 'page': 1, 'page_label': '2'}, page_content='Funded under USAID Cooperative Agreement No. 663-A-00-00-0358-00. \n \n \nProduced in collaboration with the Ethiopia Public Health Training Initiative, The Carter \nCen

In [45]:
from typing import List
from langchain.schema import Document

def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    
    minimal_docs: List[Document] = []
    for doc in docs:
        src = doc.metadata.get("Source", "")
        minimal_docs.append(
            Document(
                page_content=doc.page_content,
                metadata={"source": src}
            )
        )
    return minimal_docs

In [31]:
minimal_docs = filter_to_minimal_docs(extracted_data)



In [32]:
len(minimal_docs)

428

In [33]:
def text_split(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
    )
    texts_chunk = text_splitter.split_documents(minimal_docs)
    return texts_chunk

In [34]:
texts_chunk = text_split(minimal_docs)
print(f"Number of Chunks: {len(texts_chunk)}")

Number of Chunks: 1053


In [42]:
from langchain.embeddings import HuggingFaceEmbeddings


def download_embedding():
    
    model_name= "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(
        model_name = model_name,
        
    )
    return embeddings


embedding = download_embedding()

In [36]:
vector = embedding.embed_query("Hello world")

len(vector)

384

In [37]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [38]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY

In [39]:
from pinecone import Pinecone
pinecone_api_key = PINECONE_API_KEY

pc = Pinecone(api_key=pinecone_api_key)

In [69]:
pc

In [40]:
from pinecone import ServerlessSpec

index_name = "medical-chatbot"

if not pc.has_index(index_name):
    pc.create_index(
        name = index_name,
        dimension = 384,
        metric = "cosine",
        spec = ServerlessSpec(cloud="aws", region="us-east-1")
    )
    
index = pc.Index(index_name)

In [47]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=texts_chunk,
    embedding=embedding,
    index_name = index_name
)

PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Mon, 08 Sep 2025 20:29:44 GMT', 'Content-Type': 'application/json', 'Content-Length': '134', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '8379', 'x-pinecone-request-id': '652315941931303507', 'x-envoy-upstream-service-time': '37', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Metadata value must be a string, number, boolean or list of strings, got 'null' for field 'source'","details":[]}
